In [ ]:
import allel
import numpy as np
import pandas as pd
import polars as pl

In [ ]:
import yaml

# Read config file
configfile = "/master/abagwell/workspace/github_project/variant-analysis/config/rhesus_old.yaml"
with open(configfile, 'r') as file:
    config = yaml.safe_load(file)

# Read chromosomes
with open(config["resources"] + "ref_fna/chromosomes.list") as f:
    chromosomes = f.read().splitlines()

name = "U42_WES.founders2"

In [ ]:
# callsets = []
# for chrom in chromosomes:
vcf = config["results"] + f"genotypes/pass/{name}.SNP.autosomal.vcf.gz"
#vcf = config["results"] + "genotypes/pass/U42_WES.all2.SNP.autosomal.simplified.vcf.gz"

callset = allel.read_vcf(vcf, ['variants/CHROM', 'samples', 'calldata/GT'])


# df = pd.DataFrame()
# for chrom in range(1, 21):
#     vcf = f"/master/abagwell/variant-analysis/results/rhesus/genotypes/pass/U42_WES.founders.SNP.chr{chrom}.vcf.gz"
#     callset = allel.read_vcf(vcf, ['variants/CHROM', 'samples', 'calldata/GT'])
#     df = pd.concat([df, callset])


In [ ]:
callset["calldata/GT"]

In [ ]:
# Set numpy seed
numpy_seed = 889
np.random.seed(numpy_seed)

#chrom_filter = np.equal(callset['variants/CHROM'], '19')

# Take genotypes from current chromosome
#genotypes = allel.GenotypeArray(callset['calldata/GT'][chrom_filter])
genotypes = allel.GenotypeArray(callset['calldata/GT'])

# Allele counts
ac = genotypes.count_alleles()[:]

# Keep only biallelic SNPs
filter = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
genotypes_filtered = genotypes.compress(filter, axis=0)

# Make into 2D matrix where cells are number of non-reference alleles
genotypes_counts = genotypes_filtered.to_n_alt()

In [ ]:
ac

In [ ]:
genotypes_counts

In [ ]:
def plot_ld(genotypes_counts, title):
    m = allel.rogers_huff_r(genotypes_counts) ** 2
    ax = allel.plot_pairwise_ld(m)
    ax.set_title(title)

plot_ld(genotypes_counts[:120], 'Figure 1. Pairwise LD.')

In [ ]:
# Downsample number of SNPs used
n = 100000  # number of SNPs to choose randomly

vidx = np.random.choice(genotypes_counts.shape[0], n, replace=False)
vidx.sort()
gnr = genotypes_counts.take(vidx, axis=0)

plot_ld(gnr[:120], 'Figure 1. Pairwise LD.')

In [ ]:
# LD pruning
def ld_prune(genotypes_counts, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(genotypes_counts, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = genotypes_counts.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        genotypes_counts= genotypes_counts.compress(loc_unlinked, axis=0)
    return genotypes_counts

gnu = ld_prune(gnr, size=500, step=200, threshold=.1, n_iter=5)

plot_ld(gnu[:120], 'Figure 1. Pairwise LD.')

In [ ]:
n_components = 164
coords1, model1 = allel.pca(gnu, n_components=n_components, scaler='patterson')

In [ ]:
df = pd.DataFrame(coords1, columns = [f"PC{n}" for n in range(1,n_components+1)])
#df["sample"] = callset["samples"]
df.insert(0, "indiv", callset["samples"])

pl.from_pandas(df).write_csv(config["results"] + "pca/scikit-allel/" + f"{name}.pca.tsv", separator="\t")

# Write explained variance
with open(config["results"] + "pca/scikit-allel/" + f"{name}.variance.tsv", "w") as f:
    for line in list(model1.explained_variance_ratio_):
        f.write(f"{line}\n")

In [ ]:
## Rerun this lower half of the notebook in different environment since `allel` is not compatible with other packages
import altair as alt
import numpy as np
import pandas as pd
import polars as pl
import yaml

# Read config file
configfile = "/master/abagwell/workspace/github_project/variant-analysis/config/rhesus_old.yaml"
with open(configfile, 'r') as file:
    config = yaml.safe_load(file)

# # Read chromosomes
# with open(config["resources"] + "ref_fna/chromosomes.list") as f:
#     chromosomes = f.read().splitlines()

# Load colors (keeping only those of the founding populations)
# TODO: Generalize for when needing to remove colors or not
colors = pl.read_csv(config["colors"], separator="\t").filter(
    pl.col("Cohort").is_in(["Conventional source", "Brooks source", "NEPRC source"])
)

# TODO: Make this not have to be repeated
numpy_seed = 889
np.random.seed(numpy_seed)
name = "U42_WES.founders2"

# Read output files from scikit-allel step
df = pl.read_csv(config["results"] + "pca/scikit-allel/" + f"{name}.pca.tsv", separator="\t", schema_overrides={"indiv": pl.String})
df_variance = pl.read_csv(config["results"] + "pca/scikit-allel/" + f"{name}.variance.tsv", has_header=False, separator="\t", new_columns=["variance"]
).with_row_index("component", offset=1
).with_columns(
    pl.col("variance").mul(100).cast(pl.String).str.slice(0,5)
)

In [ ]:
# # Add batch information
# runs_file = "/master/abagwell/variant-analysis/resources/rhesus/samples/runs.WES.U42.list"
# runs = pl.read_csv(runs_file, separator="\t", has_header=False, new_columns=["batch/run"]).with_columns(
#     batch = pl.col("batch/run").str.split("/").list.get(0),
#     #pl.col("batch/run").str.split("/").list.get(1).alias("sample"),
#     sample = pl.col("batch/run").str.split("/").list.get(1).str.split("_").list.get(0),
#     library = pl.col("batch/run").str.split("/").list.get(1).str.split("_").list.get(1),
# ).with_columns(
#     sample_library = pl.concat_str([pl.col("sample"), pl.col("library")], separator="_"),
#     indiv = pl.col("sample").str.slice(3)
# ).group_by("indiv").agg(pl.first("*"))


In [ ]:
# # Add batch information
runs = pl.read_csv(config["runs"], separator="\t", schema_overrides={"indiv": pl.String})

In [ ]:
# Add colony info
# Source 1

colonies_file = config["resources"] + "pop/MML_groups_from_Martha.fixed7.tsv"
colonies = pl.read_csv(colonies_file, separator="\t", infer_schema_length=None)#.join(dates, on="Id", how="left")


# df2 = pl.from_pandas(df).join(
#     runs, how="inner", left_on="indiv", right_on="indiv"
# ).select("PC1", "PC2", "PC3", "indiv", "batch"
# )
# 

df2 = colonies.join(df, how="inner", left_on="Id", right_on="indiv")






# # Or source 2
# colonies_file = "/master/abagwell/variant-analysis/resources/rhesus/pop/colonies.tsv"
# colonies = pl.read_csv(colonies_file, separator="\t", infer_schema_length=None)

# df2 = pl.from_pandas(df).join(runs, how="inner", left_on="sample", right_on="sample_library"
# ).select("PC1", "PC2", "PC3", "sample", "batch").with_columns(
#     Id = pl.col("sample").str.split("_").list.get(0).str.slice(3)
# ).join(colonies, how="left", on="Id")

# # merged = colony_demographics.join(Q, how="left", left_on="Id", right_on="#sample"
# # ).group_by("Year", "Colony").agg(pl.count("Id").alias("Count"), pl.mean("Indian"), pl.mean("Chinese")).drop_nulls()

In [ ]:
## Plot PCA

brush = alt.selection_interval()

PC1_PC2 = alt.Chart(df2).mark_circle().encode(
    alt.X("PC1", title=f"PC1 ({df_variance.filter(pl.col("component") == 1)['variance'][0]}%)"),
    alt.Y("PC2", title=f"PC2 ({df_variance.filter(pl.col("component") == 2)['variance'][0]}%)"),
    #color=alt.condition(brush, "sample", alt.value("lightgray")),
    #color=alt.Color("batch:N", title="Batch"),
    color=alt.Color("Interval:N", title="Cohort").scale(
        domain = list(colors["Cohort"]),
        range = list(colors["Color"])
    ),
    tooltip=[
        alt.Tooltip("Id", title="Indiv")
    ],
)

PC2_PC3 = alt.Chart(df2).mark_circle().encode(
    alt.Y("PC2", title=f"PC2 ({df_variance.filter(pl.col("component") == 2)['variance'][0]}%)"),
    alt.X("PC3", title=f"PC3 ({df_variance.filter(pl.col("component") == 3)['variance'][0]}%)"),
    #color=alt.Color("batch:N", title="Batch"),
    color=alt.Color("Interval:N", title="Cohort"),
    tooltip=[
        alt.Tooltip("Id", title="Indiv")
    ],
)

PC1_PC4 = alt.Chart(df2).mark_circle().encode(
    alt.X("PC1", title=f"PC1 ({df_variance.filter(pl.col("component") == 1)['variance'][0]}%)"),
    alt.Y("PC4", title=f"PC4 ({df_variance.filter(pl.col("component") == 4)['variance'][0]}%)"),
    #color=alt.Color("batch:N", title="Batch"),
    color=alt.Color("Interval:N", title="Cohort"),
    tooltip=[
        alt.Tooltip("Id", title="Indiv")
    ],
)

# ((first) & (third | fifth)).add_params(
#     brush
# ).properties(
#     title="Rhesus PCA",
# ).configure_title(
#     anchor="middle"
# )

(PC1_PC2).add_params(
#(PC1_PC2 | PC2_PC3).add_params(
    brush
).properties(
    title="PCA of Cohorts",
).configure_title(
    anchor="middle"
#)#.save(f'/master/abagwell/figures/pca/U42.founders2.by_batch.pca{numpy_seed}.autosomal.html')
)#.save(f'/master/abagwell/figures/final_plots/U42.founders2.by_origin_PC1-PC2.pca{numpy_seed}.autosomal.html')

In [ ]:
# # TODO: Don't duplicate this line
# n_components = 164


# variance = pd.DataFrame(
#     {
#         "n_pc": range(1, n_components+1),
#         "variance": model1.explained_variance_ratio_ * 100,
#     }
# )
# variance["variance"].sum()

In [ ]:
alt.Chart(variance).mark_line(point=alt.OverlayMarkDef(filled=False, fill="white")).encode(
    alt.X("n_pc", title="nth PC"),
    alt.Y("variance", title="Percent of variance"),
    tooltip=[
        alt.Tooltip("variance", title="Variance")
    ]
).properties(
    width=900,
    title="Principle Components by Percent of Variance"
)#.save(f'/master/abagwell/figures/pca/U42_founders2.pca{numpy_seed}.autosomal.variance.svg')

In [ ]:
## Plot PCA
import altair as alt

brush = alt.selection_interval()

first = alt.Chart(df).mark_circle().encode(
    alt.X("PC1", title="PC1"),
    alt.Y("PC2", title="PC2"),
    color=alt.condition(brush, "PC1", alt.value("lightgray")),
    # tooltip=[
    # alt.Tooltip("column_1", title="Sample")
    # ],
)

second = alt.Chart(df).mark_circle().encode(
    alt.X("PC3", title="PC3"),
    alt.Y("PC4", title="PC4"),
    color=alt.condition(brush, "PC1", alt.value("lightgray"), legend=None),
    # tooltip=[
    # alt.Tooltip("column_1", title="Sample")
    # ],
)

(first | second).add_params(
    brush
).properties(
    title="Rhesus PCA",
).configure_title(
    anchor="middle"
)

In [ ]:
## UMAP
import polars.selectors as cs
import umap

max_PC = 3

#PCs = df.drop("indiv")
#PCs = df.select("PC1", "PC2", "PC3")
PCs = df2.select(cs.starts_with("PC"))[:, 0:max_PC]

reducer = umap.UMAP()
embedding = reducer.fit_transform(PCs)

umap_df = pl.from_pandas(pd.DataFrame(embedding)).insert_column(0, df2["Id"]).insert_column(0, df2["Interval"])

In [ ]:
# Plot UMAP

## Plot PCA

umap_plot = alt.Chart(umap_df).mark_circle().encode(
    alt.X("0", title=f"UMAP1"),
    alt.Y("1", title=f"UMAP2"),
    color=alt.Color("Interval:N", title="Cohort").scale(
        domain = list(colors["Cohort"]),
        range = list(colors["Color"])
    ),
    # tooltip=[
    #     alt.Tooltip("Id", title="Indiv")
    # ],
)


umap_plot.properties(
    #title="UMAP of Cohorts",
    title="UMAP of Founders",
).configure_title(
    anchor="middle"
)#.save(f'/master/abagwell/figures/pca/{name}.by_origin_PC1-PC{max_PC}.umap{numpy_seed}.autosomal.html')

In [ ]:
# Using output from `akt pca`
import altair as alt
import polars as pl

path = config["results"] + "relatedness/pca/all_samples.SNP.pca.4.tsv"


data = pl.read_csv(path, separator="\t", has_header=False)
# .filter(
#     (pl.col("column_1") == "WES30009") | (pl.col("column_1") == "WGS30009")
# )
data

In [ ]:
## Plot PCA

brush = alt.selection_interval()

first = alt.Chart(data.to_arrow().to_pandas()).mark_circle().encode(
    alt.X("column_2", title="PC1"),
    alt.Y("column_3", title="PC2"),
    color=alt.condition(brush, "column_1", alt.value("lightgray")),
    tooltip=[
    alt.Tooltip("column_1", title="Sample")
    ],
)

second = alt.Chart(data.to_arrow().to_pandas()).mark_circle().encode(
    alt.X("column_4", title="PC3"),
    alt.Y("column_5", title="PC4"),
    color=alt.condition(brush, "column_1", alt.value("lightgray"), legend=None),
    tooltip=[
    alt.Tooltip("column_1", title="Sample")
    ],
)

(first | second).add_params(
    brush
).properties(
    title="Rhesus PCA",
).configure_title(
    anchor="middle"
)#.save("rhesus_pca.html")

In [ ]:


callset = allel.read_vcf(vcf, ['variants/CHROM', 'variants/POS'])

In [ ]:
callset

In [ ]:
## SNP-density plot
import polars as pl

# TODO: Generalize chromosome lengths (currently for rhesus Mmul_10)
chromosome_lengths = [223_616_942, 196_197_964, 185_288_947, 169_963_040, 187_317_192, 179_085_566, 169_868_564, 145_679_320, 134_124_166, 99_517_758, 133_066_086, 130_043_856, 108_737_130, 128_056_306, 113_283_604, 79_627_064, 95_433_459, 74_474_043, 58_315_233, 77_137_495]  # For Mmul_10

# Read VCFs. Not that this part takes a long time to run
dfs_by_chromosome = []
for idx, chrom in enumerate(chromosomes):
    vcf = config["results"] + f"genotypes/pass/WGS/U42_WGS_WES.SNP.chr{chrom}.vcf.gz"
    df = pl.from_pandas(allel.vcf_to_dataframe(vcf, ['variants/CHROM', 'variants/POS'])).with_columns(
    pl.col("POS").floordiv(1_000_000)
    ).group_by("CHROM", "POS").agg(pl.len())
    dfs_by_chromosome.append(df)

In [ ]:
import altair as alt

plot_list = []
for chrom, chrom_len, df in zip(chromosomes, chromosome_lengths, dfs_by_chromosome):
    if chrom == "1":
        axis = alt.Axis(labels=True)
    else:
        axis = alt.Axis(labels=False, ticks=False, title="")
    plot = alt.Chart(df).mark_line().encode(
        alt.X('POS:Q', title=["Position (Gb)"], axis=axis).scale(domainMax=chrom_len/1_000_000, clamp=True),
        alt.Y('len', title=["SNP", "count"]),
        #alt.Row('variants/CHROM:O'),
    ).properties(
        #title="SNP Density",
        #title=f"chr{int(chrom)}",
        title=alt.TitleParams(f'chr{int(chrom)}', orient="left"),
        height=50,
        width=chromosome_lengths[int(chrom) - 1]/500000,
        #header=alt.Header(labelOrient='bottom'),
    )
    plot_list.append(plot.resolve_scale(y='shared'))
plot_list.reverse()


In [ ]:
alt.vconcat(*plot_list, spacing=3).configure_mark(
        #color="orange",
    )#.save("/master/abagwell/figures/roh/SNPRC.html")

In [ ]:
# Testing in a simple plot
#import altair as alt


alt.Chart(pl.concat(dfs_by_chromosome)).mark_line().encode(
    alt.X('POS:Q', title=["Position", "(Gb)"]),#.scale(domainMax=chrom_len/1_000_000, clamp=True),
    alt.Y('len', title="SNP count"),
    alt.Row('CHROM:O'),
).properties(
    #title="SNP Density",
    title=f"chr{int(chrom)}",
    height=50,
    width=800,
    #width=chromosome_lengths[int(chrom) - 1]/190000,
)

#plot_list.append(plot.resolve_scale(y='shared'))

In [ ]:
pl.concat(dfs_by_chromosome)